In [1]:
import cv2
import itertools as it
import numpy as np
import pickle
import random

from utilities import *

In [2]:
top, left, bottom, right = 609, 381, 625, 501
width, height = right - left, bottom - top

file_titles = [
    'ancient_apparition',
    'anti-mage',
    'broodmother',
    'centaur_warrunner',
    'clinkz',
    'io',
    'juggernaut',
    'keeper_of_the_light',
    "nature's_prophet",
    'nyx_assassin',
    'outworld_devourer',
    'queen_of_pain',
]
s = ''.join(c for c, _ in it.groupby(sorted(it.chain.from_iterable(file_titles))))
print(len(file_titles), s, len(s))

12 '-_abcdefghijklmnopqrstuvwxyz 29


In [3]:
def fn(file_title):
    with open(fr"D:\Dota 2\Heroes\Pickles\{file_title}.pickle", 'rb') as fin:
        return tuple(pickle_iter(fin))
data = {s: fn(s) for s in file_titles}
show_and_wait(np.vstack(list(data.items())[0][1][:33]))
cv2.destroyAllWindows()
image = list(data.items())[0][1][0]
print([len(v) for _, v in data.items()])
len(data), image.shape, image.dtype, image.min(), image.max()

[23400, 26970, 27030, 25110, 27030, 28920, 26730, 13020, 25500, 27000, 26430, 25830]


(12, (16, 120, 3), dtype('uint8'), 0, 255)

In [4]:
# Color data adds no value to this methodology.
# Transform all data into greyscale.
data = {k: tuple(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in v) for k, v in data.items()}

In [5]:
range_args = (120, 211, 15)
print(*range(*range_args))
known_thresholds = list(range(*range_args))
known_thresholds[-1] = -1

120 135 150 165 180 195 210


In [6]:
# Based on this experiment, it appears 165 is a good threshold for most of the data but, for "outworld devourer",
# this needs to be no more than 135.  Using an adaptive thresholding mechanism ensures a suitable threshold for all data.
# A given threshold excludes more background pixels in greyscale than in color.
def fn(images):
    computed_thresholds = set()
    def fn(index, threshold):
        image = images[index].copy()
        if threshold > 0:
            image[image < threshold] = 0
        else:
            threshold, image = cv2.threshold(image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)
            computed_thresholds.add(threshold)
        return image
    indices = random.sample(range(len(images)), k=99)
    show_and_wait(np.hstack([np.vstack([fn(i, n) for i in indices]) for n in known_thresholds]))
    return min(computed_thresholds), max(computed_thresholds)
g = ((k, fn(v)) for k, v in data.items())
print(*g, sep='\n')
cv2.destroyAllWindows()

('ancient_apparition', (118.0, 167.0))
('anti-mage', (96.0, 177.0))
('broodmother', (118.0, 178.0))
('centaur_warrunner', (109.0, 179.0))
('clinkz', (91.0, 178.0))
('io', (54.0, 153.0))
('juggernaut', (98.0, 181.0))
('keeper_of_the_light', (117.0, 184.0))
("nature's_prophet", (118.0, 175.0))
('nyx_assassin', (118.0, 173.0))
('outworld_devourer', (97.0, 163.0))
('queen_of_pain', (111.0, 178.0))


In [7]:
cv2.namedWindow('tesst', cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_EXPANDED)
def fn(images):
    while True:
        image = random.choice(images)
        _, image = cv2.threshold(image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)
        if show_and_wait(image) == 'g':
            break
    left_limit, right_limit = 0, image.shape[1]
    while True:
        ch = show_and_wait(np.hstack([np.ones([image.shape[0] - 7, left_limit], dtype=image.dtype) * 128,
                                      image[4:-3, left_limit:right_limit],
                                      np.ones([image.shape[0] - 7, image.shape[1] - right_limit], dtype=image.dtype) * 128]))
        if ch == 's':
            left_limit -= 1
        elif ch == 'f':
            left_limit += 1
        elif ch == 'j':
            right_limit -= 1
        elif ch == 'l':
            right_limit += 1
        elif ch == ' ':
            break
        elif ch == 'q':
            return
        else:
            print(ch, ord(ch))
    return (left_limit, right_limit)
limits = {k: fn(v) for k, v in data.items()}
cv2.destroyAllWindows()

In [8]:
# These are the limits created above.
limits = {'ancient_apparition': (14, 117),
          'anti-mage': (33, 98),
          'broodmother': (22, 110),
          'centaur_warrunner': (13, 118),
          'clinkz': (44, 87),
          'io': (58, 74),
          'juggernaut': (27, 104),
          'keeper_of_the_light': (14, 118),
          "nature's_prophet": (12, 120),
          'nyx_assassin': (25, 106),
          'outworld_devourer': (14, 117),
          'queen_of_pain': (21, 110)}

In [9]:
def fn(file_title):
    # Determine possible horizontal extents for the letters in a random sample of the images.
    horizontal_offset = limits[file_title][0]
    def fn(image):
        # Apply a threshold to and crop the image.
        _, image = cv2.threshold(image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)
        image = image[4:-3, horizontal_offset:limits[file_title][1]]

        # Collect a list of letter extents.
        l = []
        for x in range(image.shape[1] - 1):
            if not image[:, x].any() and image[:, x + 1].any():
                l.append([x + horizontal_offset])
            elif len(l) and image[:, x].any() and not image[:, x + 1].any():
                l[-1].append(x + 1 + horizontal_offset)
        return tuple(map(tuple, l))
    images = random.sample(data[file_title], k=999)
    g = map(fn, images)
    expected_length = len([c for c in file_title if c != '_'])
    s = {t for t in g if len(t) == expected_length and all(len(t) == 2 for t in t)}
    g = map(set, zip(*s))
    return tuple((min(a for a, _ in s), max(b for _, b in s), s) for s in g)
horizontal_extents = {k: fn(k) for k in data}
horizontal_extents

{'ancient_apparition': ((16, 21, {(16, 21)}),
  (21, 28, {(21, 28), (22, 28), (23, 28)}),
  (29, 34, {(29, 33), (29, 34)}),
  (35, 37, {(35, 37)}),
  (38, 43, {(38, 42), (38, 43), (39, 42)}),
  (44, 49, {(44, 49)}),
  (50, 55, {(50, 55)}),
  (58, 65, {(58, 65), (59, 65)}),
  (65, 70, {(65, 70), (66, 70)}),
  (71, 76, {(71, 76), (72, 76)}),
  (76, 83, {(76, 82), (76, 83)}),
  (83, 89, {(83, 88), (83, 89)}),
  (89, 91, {(89, 91)}),
  (91, 98, {(91, 97), (92, 97), (92, 98)}),
  (97, 101, {(97, 101), (98, 101), (99, 101)}),
  (102, 108, {(102, 108)}),
  (109, 115, {(109, 114), (109, 115)})),
 'anti-mage': ((33, 43, {(33, 42), (34, 42), (35, 41), (35, 42), (35, 43)}),
  (42, 49, {(42, 49), (43, 49)}),
  (49, 58, {(49, 56), (49, 57), (50, 56), (50, 57), (50, 58)}),
  (56, 60, {(56, 60), (57, 60), (58, 60)}),
  (61, 66, {(61, 66), (62, 65), (62, 66)}),
  (66, 74, {(66, 74), (67, 74)}),
  (74, 82, {(74, 82), (75, 82)}),
  (82, 89, {(82, 89), (83, 89)}),
  (90, 96, {(90, 95), (91, 95), (91, 96)

In [15]:
cv2.namedWindow('tesst', cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_EXPANDED)
def fn(file_title):
    images = data[file_title]
    while True:
        reference_image = random.choice(images)
        _, reference_image = cv2.threshold(reference_image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)
        ch = show_and_wait(reference_image)
        if ch == 'g':
            break
        elif ch == 'q':
            return
    selected_extents = []
    for horizontal_extent in horizontal_extents[file_title]:
        left_offset = right_offset = 0
        while True:
            image = reference_image.copy()
            for selected_extent in selected_extents:
                image[-1, selected_extent[0]:selected_extent[1]] = 128
            image[-2, horizontal_extent[0] + left_offset:horizontal_extent[1] + right_offset] = 255
            ch = show_and_wait(image)
            if ch == 's':
                left_offset -= 1
            elif ch == 'f':
                left_offset += 1
            elif ch == 'j':
                right_offset -= 1
            elif ch == 'l':
                right_offset += 1
            elif ch == ' ':
                break
            elif ch == 'q':
                return
            elif ord(ch) == 8:
                return fn(file_title)
            else:
                print(ch, ord(ch))
        selected_extents.append([horizontal_extent[0] + left_offset, horizontal_extent[1] + right_offset])
    return selected_extents
selected_extents = {k: fn(k) for k in data}
cv2.destroyAllWindows()
print(*selected_extents.items(), sep='\n')

('ancient_apparition', [[16, 22], [22, 29], [29, 35], [35, 38], [39, 43], [44, 50], [50, 56], [59, 66], [66, 71], [71, 76], [76, 83], [83, 89], [89, 92], [92, 98], [99, 102], [102, 109], [109, 115]])
('anti-mage', [[35, 43], [43, 50], [50, 57], [58, 61], [62, 66], [67, 75], [75, 83], [83, 90], [91, 96]])
('broodmother', [[24, 30], [31, 37], [37, 45], [46, 54], [54, 61], [62, 70], [71, 79], [79, 86], [87, 94], [95, 100], [101, 108]])
('centaur_warrunner', [[15, 21], [21, 26], [26, 33], [33, 39], [39, 44], [44, 50], [50, 56], [59, 68], [68, 73], [74, 79], [80, 85], [86, 92], [92, 98], [99, 105], [105, 110], [110, 116]])
('clinkz', [[46, 53], [53, 59], [59, 63], [64, 71], [72, 79], [79, 85]])
('io', [[60, 63], [64, 72]])
('juggernaut', [[29, 34], [35, 42], [43, 50], [51, 58], [58, 64], [65, 71], [72, 79], [80, 88], [88, 95], [95, 102]])
('keeper_of_the_light', [[16, 22], [22, 27], [27, 32], [33, 38], [38, 43], [44, 50], [53, 60], [60, 65], [68, 74], [74, 80], [80, 85], [89, 94], [94, 97],